# This is an example notebook on how to use NAM for texture Transfer
In this notebook we consider different refinement with the spectral embeddings


In [ ]:
import os
os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs
from geomfum.shape.mesh import TriangleMesh
from geomfum.refine import ZoomOut

import torch
import numpy as np

import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from model.neural_adjoint_map import NeuralAdjointMap
from sklearn.neighbors import NearestNeighbors

from model.neural_zoomout import NeuralZoomOut
from utils.texture_utils import generate_tex_coords, write_obj_pair

In [ ]:
import os
from urllib.request import urlretrieve

faust_url = "https://raw.githubusercontent.com/JM-data/PyFuncMap/4bde4484c3e93bff925a6a82da29fa79d6862f4b/FAUST_shapes_off/"
shape_files = ["tr_reg_080.off", "tr_reg_093.off"]
for fname in shape_files:
    url = faust_url + fname
    out_path = os.path.join("../data/", fname)
    urlretrieve(url, out_path)


In [ ]:
mesh1 = TriangleMesh.from_file("../data/tr_reg_080.off")
mesh2 = TriangleMesh.from_file("../data/tr_reg_093.off")

eigvals1, eigvecs1 = mesh1.laplacian.find_spectrum(spectrum_size=200)
eigvals2, eigvecs2 = mesh2.laplacian.find_spectrum(spectrum_size=200)

In [ ]:
p2p_gt = np.arange(mesh1.n_vertices)

In [ ]:
# create texture map
v_1 = generate_tex_coords(gs.to_numpy(mesh1.vertices))
v_2 = generate_tex_coords(gs.to_numpy(mesh2.vertices))


(v_1.shape, v_2.shape)

In [ ]:
import polyscope as ps
import matplotlib.pyplot as plt

tex = plt.imread("../texture.png")

ps.init()

# Register the mesh
mesh_ps = ps.register_surface_mesh("mesh1", mesh1.vertices.numpy(), mesh1.faces.numpy())

# Add parameterization quantity (UV coordinates)
mesh_ps.add_parameterization_quantity("texture_coords", v_1, 
                                      defined_on='vertices', enabled=False)

# Add the texture using the parameterization
mesh_ps.add_color_quantity("texture", tex[:,:,:3], 
                          defined_on='texture', 
                          param_name="texture_coords", 
                          filter_mode='linear',  # or 'nearest' for crisp textures
                          enabled=True)

ps.show()

# Initial Map

In [ ]:
from geomfum.convert import FmFromP2pConverter, P2pFromFmConverter


fmap_from_p2p=FmFromP2pConverter()
p2p_from_fmap = P2pFromFmConverter()



mesh1.basis.use_k=20
mesh2.basis.use_k=20

fmap = fmap_from_p2p(p2p_gt, mesh1.basis, mesh2.basis)
p2p_ini = p2p_from_fmap(fmap, mesh1.basis, mesh2.basis)


print(
    "Conversion Error:",
    gs.mean((mesh2.vertices[p2p_ini] - mesh2.vertices) ** 2),
)

In [ ]:
texture_transfer_map = mesh2.basis.vecs @ fmap @ mesh1.basis.pinv

v_2_transf = texture_transfer_map @ v_1

# ZoomOut

In [ ]:
mesh1.basis.use_k=200
mesh2.basis.use_k=200
zoomout = ZoomOut(nit=9, step=20)
ref_fmap = zoomout(fmap, mesh1.basis, mesh2.basis)

In [ ]:
p2p = p2p_from_fmap(ref_fmap, mesh1.basis, mesh2.basis)

print(
    "Conversion Error:",
    gs.mean((mesh2.vertices[p2p] - mesh2.vertices) ** 2),
)

In [ ]:
texture_transfer_map = mesh2.basis.vecs @ ref_fmap @ mesh1.basis.pinv

v_2_transf = texture_transfer_map @ v_1

In [ ]:
import polyscope as ps
import matplotlib.pyplot as plt

tex = plt.imread("../texture.png")

ps.init()

mesh_ps = ps.register_surface_mesh("mesh1", mesh2.vertices.numpy(), mesh2.faces.numpy())

mesh_ps.add_parameterization_quantity("texture_coords", v_2_transf, 
                                      defined_on='vertices', enabled=False)

mesh_ps.add_color_quantity("texture", tex[:,:,:3], 
                          defined_on='texture', 
                          param_name="texture_coords", 
                          filter_mode='linear',  # or 'nearest' for crisp textures
                          enabled=True)

ps.show()

# NeuralZoomOut

In [ ]:
from model.neural_zoomout import NamFromP2pConverter, P2pFromNamConverter, NeuralZoomOut

nam_from_p2p=NamFromP2pConverter()
p2p_from_nam = P2pFromNamConverter()

mesh1.basis.use_k=20
mesh2.basis.use_k=20
nam=nam_from_p2p(p2p_gt, mesh1.basis, mesh2.basis)


mesh1.basis.use_k=200   
mesh2.basis.use_k=200
nzo = NeuralZoomOut(nit=9, step=20)
ref_nam = nzo(nam, mesh1.basis, mesh2.basis)
p2p = p2p_from_nam(ref_nam, mesh1.basis, mesh2.basis)
print(
    "Conversion Error:",
    gs.mean((mesh2.vertices[p2p] - mesh2.vertices[p2p_gt]) ** 2),
)

In [ ]:
fmap = fmap_from_p2p(p2p, mesh1.basis, mesh2.basis)

texture_transfer_map = mesh2.basis.vecs @ ref_fmap @ mesh1.basis.pinv

v_2_transf = texture_transfer_map @ v_1


In [ ]:
import polyscope as ps
import matplotlib.pyplot as plt

tex = plt.imread("../texture.png")

ps.init()

mesh_ps = ps.register_surface_mesh("mesh1", mesh2.vertices.numpy(), mesh2.faces.numpy())

mesh_ps.add_parameterization_quantity("texture_coords", v_2_transf, 
                                      defined_on='vertices', enabled=False)

mesh_ps.add_color_quantity("texture", tex[:,:,:3], 
                          defined_on='texture', 
                          param_name="texture_coords", 
                          filter_mode='linear',  # or 'nearest' for crisp textures
                          enabled=True)

ps.show()